In [152]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [153]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
print(len(words), 'words loaded')
print(max(len(w) for w in words), 'max length')
print(words[:8])

29681 words loaded
15 max length
['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']


In [154]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [155]:
# Shuffle the words
import random
random.seed(42)
random.shuffle(words)

In [156]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y


n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% of the data
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% of the data
Xte, Yte = build_dataset(words[n2:]) # 10% of the data

torch.Size([170379, 8]) torch.Size([170379])
torch.Size([21222, 8]) torch.Size([21222])
torch.Size([21124, 8]) torch.Size([21124])


In [157]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '---->', itos[y.item()])

........ ----> g
.......g ----> o
......go ----> o
.....goo ----> d
....good ----> n
...goodn ----> e
..goodne ----> s
.goodnes ----> s
goodness ----> .
........ ----> w
.......w ----> e
......we ----> y
.....wey ----> l
....weyl ----> o
...weylo ----> n
..weylon ----> .
........ ----> j
.......j ----> u
......ju ----> b
.....jub ----> a


In [158]:
# original network
# n_embd = 10 # the dimensionality of the character embedding vectors
# n_hidden = 300 # the number of neurons in the hidden layer of the MLP
# model = Sequential([
#   Embedding(vocab_size, n_embd),
#   FlattenConsecutive(8), Linear(n_embd * 8, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
#   Linear(n_hidden, vocab_size),
# ])

# hierarchical network
n_embd = 24 # the dimensionality of the character embedding vectors
n_hidden = 128 # the number of neurons in the hidden layer of the MLP
model = Sequential([
  Embedding(vocab_size, n_embd),
  FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, vocab_size),
])

# parameter init
with torch.no_grad():
  model.layers[-1].weight *= 0.1 # last layer make less confident

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

76579


In [159]:
ix = torch.randint(0, Xtr.shape[0], (4,)) # looking at bactch of 4 examples
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
Xb

torch.Size([4, 8])


tensor([[13,  9, 12, 12,  9,  1, 14, 15],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 10,  1, 13,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [160]:
model.layers[0].out.shape # output of the embedding layer

torch.Size([4, 8, 24])

In [161]:
model.layers[1].out.shape # output of the first linear layer

torch.Size([4, 4, 48])

In [162]:
model.layers[2].out.shape # output of the second linear layer

torch.Size([4, 4, 128])

In [163]:
torch.manual_seed(42); # seed rng for reproducibility

In [176]:
list(range(10))[::2]

[0, 2, 4, 6, 8]

In [179]:
e = torch.randn(4, 8, 10) # embeddings
explicit = torch.cat([e[:, :4, :], e[:, 4:, :]], dim=2) # concatenate along the last dimension
explicit.shape

torch.Size([4, 4, 20])

In [165]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix]  # batch X, Y
  
  # Convert Xb (indices) into embeddings
  Xb = Xb.long()  # Cast indices to long to avoid indexing errors
  
  # Flatten embeddings for the model's input
  Xb = Xb.view(Xb.shape[0], -1)  # shape: [batch_size, block_size * n_embd]
  
  # forward pass
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb)  # loss function
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update: simple SGD
  lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0:  # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  break


      0/  50000: 2.6316


In [166]:
print(f'Xb shape: {Xb.shape}, Yb shape: {Yb.shape}')
print(f'Logits shape: {logits.shape}')

Xb shape: torch.Size([32, 8]), Yb shape: torch.Size([32])
Logits shape: torch.Size([32, 27])


In [167]:
# plt.plot(lossi)

In [168]:
# plt.plot(torch.tensor(lossi)).view(-1, 1000).mean(1)

In [169]:
# put layers into eval mode (needed for batchnorm especially)
for layer in model.layers:
  layer.training = False

In [170]:
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking inside pytorch
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.2131175994873047
val 2.2173640727996826


In [171]:
# sample from the model
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      logits = model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out)) # decode and print the generated word

kisgono.
eriian.
mersath.
aamyah.
zvayn.
jaiar.
tymon.
kalton.
kyanna.
isansulia.
ianon.
shma.
nivxann.
kaumria.
ri.
lovary.
doudn.
chsanna.
bavalore.
nuliani.


In [172]:

for x,y in zip(Xtr[7:15], Ytr[7:15]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

.goodnes --> s
goodness --> .
........ --> w
.......w --> e
......we --> y
.....wey --> l
....weyl --> o
...weylo --> n


In [173]:
# forward a single example:
logits = model(Xtr[[7]])
logits.shape

torch.Size([1, 27])

In [174]:
# forward all of them
logits = torch.zeros(8, 27)
for i in range(8):
  logits[i] = model(Xtr[[7+i]])
logits.shape

torch.Size([8, 27])

In [175]:
# convolution is a "for loop"
# allows us to forward Linear layers efficiently over space